# 📚 **Tarea Integradora: Análisis y Desarrollo de un Agente de Aprendizaje por Refuerzo**

---

## 🎯 **Información General**

**Curso**: Introducción a la Programación en Python para Aprendizaje por Refuerzo  
**Modalidad**: Google Colaboratory  
**Tiempo estimado**: 4-6 horas  
**Puntuación total**: 100 puntos  

---

## 📋 **Objetivos de la Actividad**

- **Integrar conceptos fundamentales** de Python: estructuras de datos, POO, manipulación con Pandas/NumPy
- **Implementar algoritmos básicos de RL**: Desarrollar un agente Q-Learning simplificado
- **Analizar resultados experimentales**: Procesar métricas de rendimiento y generar visualizaciones
- **Demostrar competencias** en el ecosistema científico de Python

---

## 🔧 **Configuración del Entorno**

In [ ]:
# Importación de bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, deque
import random
from typing import Tuple, List, Dict
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_palette("husl")

# Semilla para reproducibilidad
np.random.seed(42)
random.seed(42)

print("✅ Bibliotecas importadas correctamente")
print("📊 Configuración de visualización establecida")
print("🎲 Semilla aleatoria fijada para reproducibilidad")

---

## 📝 **PARTE A: Implementación del Entorno y Agente (40 puntos)**

### 🌐 **Paso 1: Implementar GridWorldEnvironment**

**Especificaciones técnicas:**
- Cuadrícula 4×4 (16 estados: 0-15)
- Estado inicial: 0 (esquina superior izquierda)
- Objetivo: estado 15 (esquina inferior derecha)
- Obstáculos: estados [5, 7, 11]
- Acciones: 0=↑, 1=→, 2=↓, 3=←
- Recompensas: +10 (objetivo), -1 (obstáculo), -0.1 (movimiento normal)

**Representación del grid:**

In [ ]:
# Implementación del entorno GridWorld
class GridWorldEnvironment:
    """
    Entorno de cuadrícula 4x4 para aprendizaje por refuerzo.
    """

    def __init__(self):
        # TODO: Implementar inicialización
        # Definir: grid_size, n_states, n_actions, obstacles, goal_state, current_state
        self.grid_size = 4
        self.n_states = 16
        self.n_actions = 4
        self.obstacles = [5, 7, 11]
        self.goal_state = 15
        self.current_state = 0

        # Definir movimientos: 0=↑, 1=→, 2=↓, 3=←
        self.actions = {
            0: (-1, 0),  # arriba
            1: (0, 1),   # derecha
            2: (1, 0),   # abajo
            3: (0, -1)   # izquierda
        }

        print(f"🌐 GridWorld 4x4 inicializado")
        print(f"   Estados: {self.n_states}")
        print(f"   Obstáculos: {self.obstacles}")
        print(f"   Objetivo: {self.goal_state}")

    def reset(self) -> int:
        """
        Reinicia el entorno al estado inicial.
        Returns:
            int: Estado inicial (0)
        """
        # TODO: Implementar reset
        self.current_state = 0
        return self.current_state

    def step(self, action: int) -> Tuple[int, float, bool, dict]:
        """
        Ejecuta una acción en el entorno.

        Args:
            action (int): Acción a ejecutar (0=↑, 1=→, 2=↓, 3=←)

        Returns:
            Tuple: (next_state, reward, done, info)
        """
        # TODO: Implementar lógica de transición
        # Considerar: validación de fronteras, detección de obstáculos, cálculo de recompensas

        # Obtener coordenadas actuales
        row, col = self.state_to_coordinates(self.current_state)

        # Calcular nueva posición
        delta_row, delta_col = self.actions[action]
        new_row = row + delta_row
        new_col = col + delta_col

        # Validar fronteras
        if new_row < 0 or new_row >= self.grid_size or new_col < 0 or new_col >= self.grid_size:
            # Fuera de fronteras, mantener posición actual
            next_state = self.current_state
        else:
            next_state = self.coordinates_to_state(new_row, new_col)

        # Calcular recompensa
        if next_state == self.goal_state:
            reward = 10.0
            done = True
        elif next_state in self.obstacles:
            reward = -1.0
            done = True
        else:
            reward = -0.1
            done = False

        # Actualizar estado actual
        self.current_state = next_state

        info = {
            'coordinates': self.state_to_coordinates(next_state),
            'action_name': ['↑', '→', '↓', '←'][action]
        }

        return next_state, reward, done, info

    def get_valid_actions(self, state: int) -> List[int]:
        """
        Obtiene acciones válidas desde un estado dado.

        Args:
            state (int): Estado actual

        Returns:
            List[int]: Lista de acciones válidas
        """
        # TODO: Implementar validación de acciones
        valid_actions = []
        row, col = self.state_to_coordinates(state)

        for action in range(self.n_actions):
            delta_row, delta_col = self.actions[action]
            new_row = row + delta_row
            new_col = col + delta_col

            # Verificar si la nueva posición está dentro de fronteras
            if 0 <= new_row < self.grid_size and 0 <= new_col < self.grid_size:
                valid_actions.append(action)

        return valid_actions

    def state_to_coordinates(self, state: int) -> Tuple[int, int]:
        """Convierte estado a coordenadas (fila, columna)."""
        # TODO: Implementar conversión estado -> coordenadas
        row = state // self.grid_size
        col = state % self.grid_size
        return row, col

    def coordinates_to_state(self, row: int, col: int) -> int:
        """Convierte coordenadas a estado."""
        # TODO: Implementar conversión coordenadas -> estado
        return row * self.grid_size + col

    def render(self):
        """Visualiza el estado actual del entorno."""
        # TODO: Implementar visualización opcional
        print("\nGrid actual:")
        for row in range(self.grid_size):
            line = ""
            for col in range(self.grid_size):
                state = self.coordinates_to_state(row, col)
                if state == self.current_state:
                    line += "[A] "  # Agente
                elif state == self.goal_state:
                    line += "[G] "  # Goal
                elif state in self.obstacles:
                    line += "[X] "  # Obstáculo
                else:
                    line += "[ ] "
            print(line)
        print()

# Prueba inicial del entorno
env = GridWorldEnvironment()
print("Estado inicial:", env.reset())
print("Acciones válidas desde estado 0:", env.get_valid_actions(0))
env.render()

### 🤖 **Paso 2: Implementar SimpleQLearningAgent**

**Especificaciones del agente:**
- Algoritmo: Q-Learning con tabla tabular
- Exploración: Estrategia epsilon-greedy con decaimiento
- Actualización: Q(s,a) ← Q(s,a) + α[r + γ max Q(s',a') - Q(s,a)]
- Parámetros configurables: α, γ, ε, decaimiento de ε

In [ ]:
# Implementación del agente Q-Learning
class SimpleQLearningAgent:
    """
    Agente Q-Learning con exploración epsilon-greedy.

    Implementa la ecuación de actualización:
    Q(s,a) ← Q(s,a) + α[r + γ max Q(s',a') - Q(s,a)]
    """

    def __init__(self, n_states: int, n_actions: int,
                 learning_rate: float = 0.1, discount_factor: float = 0.95,
                 epsilon: float = 0.1, epsilon_decay: float = 0.995,
                 epsilon_min: float = 0.01):
        """
        Inicializa el agente Q-Learning.

        Args:
            n_states: Número de estados en el entorno
            n_actions: Número de acciones disponibles
            learning_rate: Tasa de aprendizaje (α)
            discount_factor: Factor de descuento (γ)
            epsilon: Probabilidad de exploración inicial
            epsilon_decay: Factor de decaimiento de epsilon
            epsilon_min: Valor mínimo de epsilon
        """
        # TODO: Implementar inicialización
        # Inicializar: tabla Q, parámetros de aprendizaje, métricas de seguimiento
        self.n_states = n_states
        self.n_actions = n_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

        # Inicializar tabla Q con ceros
        self.q_table = np.zeros((n_states, n_actions))

        # Métricas de seguimiento
        self.td_errors = []
        self.training_step = 0

        print(f"🤖 Agente Q-Learning inicializado")
        print(f"   Estados: {n_states}, Acciones: {n_actions}")
        print(f"   α: {learning_rate}, γ: {discount_factor}")
        print(f"   ε inicial: {epsilon}, decaimiento: {epsilon_decay}")

    def choose_action(self, state: int, training: bool = True) -> int:
        """
        Selecciona acción usando estrategia epsilon-greedy.

        Args:
            state: Estado actual
            training: Si True, aplica exploración; si False, política greedy pura

        Returns:
            int: Acción seleccionada
        """
        # TODO: Implementar selección de acción epsilon-greedy
        if training and np.random.random() < self.epsilon:
            # Exploración: acción aleatoria
            return np.random.randint(0, self.n_actions)
        else:
            # Explotación: mejor acción según tabla Q
            return np.argmax(self.q_table[state])

    def update_q_table(self, state: int, action: int, reward: float,
                      next_state: int, done: bool) -> float:
        """
        Actualiza la tabla Q usando la regla de Q-Learning.

        Args:
            state: Estado actual
            action: Acción ejecutada
            reward: Recompensa observada
            next_state: Estado siguiente
            done: Si el episodio terminó

        Returns:
            float: Error de diferencia temporal (TD error)
        """
        # TODO: Implementar actualización Q-Learning
        # Calcular: Q_target, TD_error, actualizar Q(s,a)

        # Valor Q actual
        current_q = self.q_table[state, action]

        # Calcular target
        if done:
            target_q = reward
        else:
            target_q = reward + self.discount_factor * np.max(self.q_table[next_state])

        # Error de diferencia temporal
        td_error = target_q - current_q

        # Actualizar tabla Q
        self.q_table[state, action] += self.learning_rate * td_error

        # Registrar métricas
        self.td_errors.append(abs(td_error))
        self.training_step += 1

        return abs(td_error)

    def decay_epsilon(self):
        """Reduce epsilon gradualmente."""
        # TODO: Implementar decaimiento de epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def get_policy(self) -> np.ndarray:
        """
        Extrae política determinística de la tabla Q.

        Returns:
            np.ndarray: Matriz de política [n_states x n_actions]
        """
        # TODO: Implementar extracción de política
        policy = np.zeros((self.n_states, self.n_actions))
        for state in range(self.n_states):
            best_action = np.argmax(self.q_table[state])
            policy[state, best_action] = 1.0
        return policy

    def get_value_function(self) -> np.ndarray:
        """
        Calcula función de valor V(s) = max_a Q(s,a).

        Returns:
            np.ndarray: Vector de función de valor
        """
        # TODO: Implementar cálculo de función de valor
        return np.max(self.q_table, axis=1)

# Prueba inicial del agente
agent = SimpleQLearningAgent(n_states=16, n_actions=4, epsilon=0.3)
print("Acción inicial desde estado 0:", agent.choose_action(0))
print("Forma de la tabla Q:", agent.q_table.shape)
print("Función de valor inicial:", agent.get_value_function()[:5])

---

## 🧪 **PARTE B: Experimentación y Análisis de Datos (35 puntos)**

### 📊 **Paso 3: Script de Entrenamiento y Recolección de Datos**

En esta sección implementaremos el loop de entrenamiento que:
- Ejecuta 1000 episodios de entrenamiento
- Registra métricas de rendimiento por episodio
- Aplica decaimiento gradual de epsilon
- Genera reportes de progreso periódicos

In [ ]:
# Función de entrenamiento
def train_agent(env, agent, n_episodes: int = 1000, max_steps: int = 100) -> pd.DataFrame:
    """
    Entrena el agente y recolecta métricas de rendimiento.

    Args:
        env: Entorno de entrenamiento
        agent: Agente a entrenar
        n_episodes: Número de episodios
        max_steps: Máximo de pasos por episodio

    Returns:
        pd.DataFrame: Métricas de entrenamiento
    """

    # Contenedores para métricas
    metrics = {
        'episode': [],
        'reward': [],
        'steps': [],
        'epsilon': [],
        'states_visited': [],
        'td_errors': []
    }

    print(f"🚀 Iniciando entrenamiento: {n_episodes} episodios")
    print("-" * 50)

    for episode in range(n_episodes):
        # TODO: Implementar loop de entrenamiento
        # 1. Reset del entorno
        state = env.reset()

        # 2. Loop del episodio hasta terminar o max_steps
        episode_reward = 0
        episode_steps = 0
        episode_states = set()
        episode_td_errors = []

        for step in range(max_steps):
            # Seleccionar acción
            action = agent.choose_action(state, training=True)

            # Ejecutar acción
            next_state, reward, done, info = env.step(action)

            # Actualizar agente
            td_error = agent.update_q_table(state, action, reward, next_state, done)

            # Registrar métricas del paso
            episode_reward += reward
            episode_steps += 1
            episode_states.add(state)
            episode_td_errors.append(td_error)

            # Preparar siguiente iteración
            state = next_state

            if done:
                break

        # 3. Registrar métricas del episodio
        metrics['episode'].append(episode)
        metrics['reward'].append(episode_reward)
        metrics['steps'].append(episode_steps)
        metrics['epsilon'].append(agent.epsilon)
        metrics['states_visited'].append(len(episode_states))
        metrics['td_errors'].append(np.mean(episode_td_errors) if episode_td_errors else 0)

        # 4. Aplicar decaimiento de epsilon
        agent.decay_epsilon()

        # 5. Reportar progreso cada 100 episodios
        if (episode + 1) % 100 == 0:
            avg_reward = np.mean(metrics['reward'][-100:])
            avg_steps = np.mean(metrics['steps'][-100:])
            print(f"Episodio {episode + 1:4d} | "
                  f"Recompensa promedio: {avg_reward:6.2f} | "
                  f"Pasos promedio: {avg_steps:5.1f} | "
                  f"Epsilon: {agent.epsilon:.3f}")

    # TODO: Crear DataFrame con métricas
    df = pd.DataFrame(metrics)
    df['moving_average'] = df['reward'].rolling(window=50, min_periods=1).mean()

    print(f"\n✅ Entrenamiento completado")
    print(f"📊 Datos recolectados: {len(df)} episodios")

    return df

# Ejecución del entrenamiento
print("🎮 Creando nuevo entorno y agente para entrenamiento...")
env = GridWorldEnvironment()
agent = SimpleQLearningAgent(n_states=16, n_actions=4,
                           learning_rate=0.1,
                           epsilon=0.3,
                           epsilon_decay=0.995)

# Ejecutar entrenamiento
training_data = train_agent(env, agent, n_episodes=1000)

### 📈 **Paso 4: Procesamiento y Análisis de Datos**

Ahora analizaremos estadísticamente los datos recolectados durante el entrenamiento:
- Estadísticas descriptivas básicas
- Detección de convergencia
- Análisis de distribuciones
- Correlaciones entre variables

In [ ]:
# Análisis estadístico de los datos
def analyze_training_data(df: pd.DataFrame) -> Dict:
    """
    Realiza análisis estadístico comprehensivo de los datos de entrenamiento.

    Args:
        df: DataFrame con métricas de entrenamiento

    Returns:
        Dict: Estadísticas de análisis
    """

    analysis = {}

    # TODO: Implementar análisis estadístico
    # 1. Estadísticas descriptivas básicas
    analysis['total_episodes'] = len(df)
    analysis['final_average_reward'] = df['reward'].tail(100).mean()
    analysis['best_episode_reward'] = df['reward'].max()
    analysis['worst_episode_reward'] = df['reward'].min()

    # 2. Tasa de convergencia (episodio donde se estabiliza)
    # Considerar convergencia cuando la media móvil se estabiliza
    moving_avg = df['moving_average'].values
    differences = np.abs(np.diff(moving_avg))
    convergence_threshold = 0.1

    convergence_episode = None
    for i in range(100, len(differences)):
        if np.all(differences[i-20:i] < convergence_threshold):
            convergence_episode = i
            break

    analysis['convergence_episode'] = convergence_episode if convergence_episode else "No detectada"

    # 3. Análisis de cuartiles para longitud de episodios
    steps_quartiles = np.percentile(df['steps'], [25, 50, 75])
    analysis['steps_q1'] = steps_quartiles[0]
    analysis['steps_median'] = steps_quartiles[1]
    analysis['steps_q3'] = steps_quartiles[2]

    # 4. Correlaciones entre variables
    correlation_matrix = df[['reward', 'steps', 'epsilon', 'states_visited']].corr()
    analysis['reward_steps_correlation'] = correlation_matrix.loc['reward', 'steps']
    analysis['reward_epsilon_correlation'] = correlation_matrix.loc['reward', 'epsilon']

    # 5. Estadísticas de exploración
    analysis['initial_epsilon'] = df['epsilon'].iloc[0]
    analysis['final_epsilon'] = df['epsilon'].iloc[-1]
    analysis['average_states_visited'] = df['states_visited'].mean()

    # 6. Tendencias de mejora
    first_half_reward = df['reward'].iloc[:len(df)//2].mean()
    second_half_reward = df['reward'].iloc[len(df)//2:].mean()
    analysis['improvement_percentage'] = ((second_half_reward - first_half_reward) / abs(first_half_reward)) * 100

    return analysis

# Ejecutar análisis
analysis_results = analyze_training_data(training_data)

print("📊 ANÁLISIS ESTADÍSTICO DE ENTRENAMIENTO")
print("=" * 50)
for key, value in analysis_results.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

# Mostrar estadísticas descriptivas del DataFrame
print("\n📈 ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 30)
print(training_data[['reward', 'steps', 'epsilon', 'states_visited']].describe())

---

## 📊 **PARTE C: Visualización y Interpretación (25 puntos)**

### 🎨 **Paso 5: Generación de Visualizaciones**

Crearemos un conjunto comprehensivo de visualizaciones que incluye:
1. **Curva de aprendizaje**: Recompensa por episodio + media móvil
2. **Eficiencia del agente**: Evolución de pasos por episodio
3. **Exploración**: Decaimiento de epsilon durante entrenamiento
4. **Mapa de calor**: Política final aprendida en la cuadrícula 4x4

In [ ]:
# Función de visualización comprehensiva
def create_comprehensive_plots(df: pd.DataFrame, agent, env):
    """
    Genera visualizaciones comprehensivas del proceso de entrenamiento.

    Args:
        df: DataFrame con métricas de entrenamiento
        agent: Agente entrenado
        env: Entorno utilizado
    """

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # TODO: Implementar cada subgráfico

    # 1. Curva de aprendizaje (superior izquierdo)
    axes[0,0].plot(df['episode'], df['reward'], alpha=0.3, color='lightblue', label='Recompensa por episodio')
    axes[0,0].plot(df['episode'], df['moving_average'], color='darkblue', linewidth=2, label='Media móvil (50 ep.)')
    axes[0,0].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='Línea base')
    axes[0,0].set_title('Curva de Aprendizaje', fontweight='bold', fontsize=12)
    axes[0,0].set_xlabel('Episodio')
    axes[0,0].set_ylabel('Recompensa Acumulada')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)

    # 2. Eficiencia del agente (superior derecho)
    axes[0,1].plot(df['episode'], df['steps'], color='green', alpha=0.6, label='Pasos por episodio')
    # Agregar línea de tendencia
    z = np.polyfit(df['episode'], df['steps'], 1)
    p = np.poly1d(z)
    axes[0,1].plot(df['episode'], p(df['episode']), color='darkgreen', linewidth=2, linestyle='--', label='Tendencia')
    axes[0,1].set_title('Eficiencia del Agente', fontweight='bold', fontsize=12)
    axes[0,1].set_xlabel('Episodio')
    axes[0,1].set_ylabel('Pasos hasta Terminación')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)

    # 3. Exploración (inferior izquierdo)
    axes[1,0].plot(df['episode'], df['epsilon'], color='orange', linewidth=2)
    axes[1,0].axhline(y=0.1, color='red', linestyle='--', alpha=0.7, label='Exploración baja')
    axes[1,0].axhline(y=0.3, color='yellow', linestyle='--', alpha=0.7, label='Exploración alta')
    axes[1,0].set_title('Evolución de la Exploración (ε)', fontweight='bold', fontsize=12)
    axes[1,0].set_xlabel('Episodio')
    axes[1,0].set_ylabel('Epsilon')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)

    # 4. Mapa de calor de política (inferior derecho)
    policy = agent.get_policy()
    policy_grid = np.zeros((4, 4))

    # Convertir política a grid 4x4 con direcciones
    action_arrows = ['↑', '→', '↓', '←']
    policy_arrows = np.empty((4, 4), dtype=object)

    for state in range(16):
        row, col = env.state_to_coordinates(state)
        best_action = np.argmax(policy[state])
        policy_grid[row, col] = best_action
        policy_arrows[row, col] = action_arrows[best_action]

    # Crear mapa de calor
    im = axes[1,1].imshow(policy_grid, cmap='viridis', alpha=0.7)

    # Agregar texto con direcciones y estados especiales
    for row in range(4):
        for col in range(4):
            state = env.coordinates_to_state(row, col)
            if state == 0:  # Estado inicial
                text = f"START\n{policy_arrows[row, col]}"
                color = 'white'
            elif state == 15:  # Objetivo
                text = "GOAL"
                color = 'yellow'
            elif state in env.obstacles:  # Obstáculos
                text = "XXX"
                color = 'red'
            else:
                text = policy_arrows[row, col]
                color = 'white'

            axes[1,1].text(col, row, text, ha='center', va='center',
                         color=color, fontweight='bold', fontsize=10)

    axes[1,1].set_title('Política Final Aprendida', fontweight='bold', fontsize=12)
    axes[1,1].set_xlabel('Columna')
    axes[1,1].set_ylabel('Fila')

    # Configurar ticks
    axes[1,1].set_xticks(range(4))
    axes[1,1].set_yticks(range(4))

    plt.tight_layout()
    plt.suptitle('Análisis Comprehensivo: Entrenamiento Q-Learning GridWorld 4x4',
                 fontsize=16, fontweight='bold', y=0.98)
    plt.show()

# Ejecutar visualización
create_comprehensive_plots(training_data, agent, env)

### 🔍 **Paso 6: Análisis Interpretativo**

**INSTRUCCIONES**: Complete el siguiente análisis con un mínimo de 200 palabras, abordando cada punto solicitado. Sustente sus observaciones con los resultados obtenidos en su entrenamiento.

---

## 📝 **ANÁLISIS E INTERPRETACIÓN DE RESULTADOS**

### **1. Patrones de Convergencia Observados:**
[TODO: Describa cómo evolucionó la recompensa a lo largo del entrenamiento. ¿En qué episodio aproximado observa estabilización? ¿Hubo fluctuaciones significativas? Mencione valores específicos de su entrenamiento.]

### **2. Efectividad de la Estrategia Epsilon-Greedy:**
[TODO: Analice cómo el balance entre exploración y explotación afectó el aprendizaje. ¿Fue apropiado el decaimiento de epsilon? ¿Qué mejoras propondría? Considere los valores inicial y final de epsilon.]

### **3. Calidad de la Política Aprendida:**
[TODO: Evalúe la política final mostrada en el mapa de calor. ¿El agente aprendió una estrategia óptima para llegar al objetivo? ¿Evita obstáculos eficientemente? ¿Hay direcciones que parecen subóptimas?]

### **4. Limitaciones del Enfoque Tabular:**
[TODO: Reflexione sobre las limitaciones de Q-Learning tabular. ¿Cómo escalaría este enfoque a problemas con espacios de estados más grandes (ej: 100x100)? ¿Qué alternativas conoce (aproximación funcional, redes neuronales)?]

### **5. Observaciones Adicionales:**
[TODO: Incluya cualquier insight adicional sobre el comportamiento del agente, patrones inesperados, relación entre métricas, o mejoras potenciales al algoritmo implementado.]

---

**Extensión mínima requerida**: 200 palabras  
**Criterios de evaluación**: Profundidad del análisis, conexión con resultados obtenidos, pensamiento crítico, propuestas de mejora

In [ ]:
# Celda para que los estudiantes escriban su análisis interpretativo
analysis_text = """
# 📋 MI ANÁLISIS INTERPRETATIVO

## 1. Patrones de Convergencia Observados
[Escriba aquí su análisis basado en los gráficos generados...]

## 2. Efectividad de la Estrategia Epsilon-Greedy
[Escriba aquí su análisis sobre la exploración vs explotación...]

## 3. Calidad de la Política Aprendida
[Escriba aquí su evaluación de la política final...]

## 4. Limitaciones del Enfoque Tabular
[Escriba aquí sus reflexiones sobre escalabilidad...]

## 5. Observaciones Adicionales
[Escriba aquí insights adicionales...]
"""

print("📝 Sección de análisis preparada")
print("✏️ Complete su análisis interpretativo arriba")
print("💡 Recuerde: mínimo 200 palabras, conectar con sus resultados específicos")

---

## 📊 **Evaluación y Métricas Finales**

### **Paso 7: Evaluación del Rendimiento Final**

Evaluaremos el agente entrenado ejecutando episodios sin exploración (ε=0) para medir su rendimiento real.

In [ ]:
# Métricas de evaluación final
def evaluate_final_performance(agent, env, n_test_episodes: int = 100) -> Dict:
    """
    Evalúa el rendimiento final del agente entrenado.

    Args:
        agent: Agente entrenado
        env: Entorno de evaluación
        n_test_episodes: Número de episodios de prueba

    Returns:
        Dict: Métricas de rendimiento final
    """

    # TODO: Implementar evaluación final
    success_count = 0
    total_rewards = []
    total_steps = []
    outcomes = []

    print(f"🎯 Evaluando rendimiento final: {n_test_episodes} episodios sin exploración")

    for episode in range(n_test_episodes):
        # Reiniciar entorno
        state = env.reset()
        episode_reward = 0
        episode_steps = 0

        # Ejecutar episodio sin exploración
        for step in range(100):  # Máximo 100 pasos
            action = agent.choose_action(state, training=False)  # Sin exploración
            next_state, reward, done, info = env.step(action)

            episode_reward += reward
            episode_steps += 1
            state = next_state

            if done:
                if reward > 0:  # Llegó al objetivo
                    success_count += 1
                    outcomes.append("Éxito")
                else:  # Cayó en obstáculo
                    outcomes.append("Obstáculo")
                break
        else:
            outcomes.append("Timeout")  # No terminó en tiempo límite

        total_rewards.append(episode_reward)
        total_steps.append(episode_steps)

    # Calcular métricas
    metrics = {
        'success_rate': success_count / n_test_episodes,
        'average_reward': np.mean(total_rewards),
        'std_reward': np.std(total_rewards),
        'average_steps': np.mean(total_steps),
        'std_steps': np.std(total_steps),
        'best_reward': np.max(total_rewards),
        'worst_reward': np.min(total_rewards),
        'success_episodes': success_count,
        'total_episodes': n_test_episodes
    }

    # Mostrar distribución de resultados
    outcome_counts = {outcome: outcomes.count(outcome) for outcome in set(outcomes)}
    metrics['outcome_distribution'] = outcome_counts

    return metrics

# Ejecutar evaluación final
final_metrics = evaluate_final_performance(agent, env, n_test_episodes=100)

print("\n🎯 MÉTRICAS DE RENDIMIENTO FINAL")
print("=" * 40)
for metric, value in final_metrics.items():
    if isinstance(value, float):
        if metric.endswith('_rate'):
            print(f"{metric}: {value:.1%}")
        else:
            print(f"{metric}: {value:.3f}")
    else:
        print(f"{metric}: {value}")

# Visualización adicional: distribución de recompensas finales
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
rewards_eval = []
steps_eval = []
for _ in range(100):
    state = env.reset()
    ep_reward = 0
    ep_steps = 0
    for step in range(100):
        action = agent.choose_action(state, training=False)
        state, reward, done, _ = env.step(action)
        ep_reward += reward
        ep_steps += 1
        if done:
            break
    rewards_eval.append(ep_reward)
    steps_eval.append(ep_steps)

plt.hist(rewards_eval, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Distribución de Recompensas (Evaluación Final)')
plt.xlabel('Recompensa por Episodio')
plt.ylabel('Frecuencia')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(steps_eval, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
plt.title('Distribución de Pasos (Evaluación Final)')
plt.xlabel('Pasos por Episodio')
plt.ylabel('Frecuencia')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📈 Recompensa promedio en evaluación: {np.mean(rewards_eval):.2f}")
print(f"📊 Pasos promedio en evaluación: {np.mean(steps_eval):.1f}")

---

## 🏆 **Criterios de Evaluación y Lista de Verificación**

### **Criterios de Evaluación**

| **Componente** | **Puntuación** | **Criterios de Evaluación** |
|----------------|----------------|------------------------------|
| **Implementación POO** | 20 pts | Clases bien estructuradas, métodos apropiados, documentación clara |
| **Algoritmo Q-Learning** | 20 pts | Implementación matemáticamente correcta, manejo apropiado de parámetros |
| **Procesamiento de Datos** | 15 pts | Uso efectivo de Pandas, análisis estadístico completo y apropiado |
| **Visualización** | 20 pts | Gráficos informativos, etiquetado apropiado, estética profesional |
| **Análisis Interpretativo** | 15 pts | Insights profundos, conexiones teóricas, pensamiento crítico |
| **Código y Documentación** | 10 pts | Estilo PEP8, comentarios explicativos, código ejecutable |

### **Escala de Calificación por Componente**

- **Excelente (90-100%)**: Implementación completa y correcta, análisis profundo, código profesional
- **Bueno (80-89%)**: Implementación correcta con pequeñas mejoras posibles, análisis sólido
- **Satisfactorio (70-79%)**: Funcionalidad básica correcta, análisis superficial pero presente
- **Insuficiente (<70%)**: Errores significativos, análisis incompleto o incorrecto

---

## ✅ **Lista de Verificación Pre-Entrega**

Marque cada item antes de entregar su trabajo:

- [ ] **Código ejecutable**: Todas las celdas se ejecutan sin errores
- [ ] **GridWorldEnvironment**: Implementación completa con todos los métodos requeridos
- [ ] **SimpleQLearningAgent**: Implementación correcta del algoritmo Q-Learning
- [ ] **Entrenamiento**: 1000 episodios ejecutados exitosamente
- [ ] **Análisis de datos**: Procesamiento completo con Pandas, estadísticas calculadas
- [ ] **Visualizaciones**: Las 4 gráficas requeridas generadas correctamente
- [ ] **Análisis interpretativo**: Mínimo 200 palabras, todos los puntos abordados
- [ ] **Evaluación final**: Métricas de rendimiento calculadas sin exploración
- [ ] **Comentarios**: Código apropiadamente comentado y documentado
- [ ] **Nomenclatura**: Archivo nombrado como `RL_Python_[Apellido]_[Nombre].ipynb`

---

## 📚 **Recursos de Referencia Adicionales**

- **Sutton & Barto (2018)**: Capítulos 1-6 para fundamentos teóricos de RL
- **Documentación NumPy**: [numpy.org/doc](https://numpy.org/doc/) - Para operaciones con arrays
- **Documentación Pandas**: [pandas.pydata.org/docs](https://pandas.pydata.org/docs/) - Para manipulación de datos
- **Documentación Matplotlib**: [matplotlib.org](https://matplotlib.org/) - Para visualización
- **PEP 8**: [pep8.org](https://pep8.org/) - Guía de estilo para Python

---

## 🎓 **Mensaje Final**

**¡Felicitaciones por completar esta tarea integradora!**

Esta actividad representa la culminación de sus aprendizajes en el curso, integrando programación orientada a objetos, manipulación de datos, visualización y fundamentos de aprendizaje por refuerzo.

El agente Q-Learning que han implementado constituye uno de los algoritmos fundamentales del campo, y las habilidades desarrolladas les proporcionan una base sólida para abordar problemas más complejos en inteligencia artificial y ciencia de datos.

**Recuerden**: La excelencia no radica únicamente en obtener resultados correctos, sino en comprender profundamente los procesos subyacentes y ser capaces de articular insights significativos sobre el comportamiento observado.

¡Éxito en su implementación! 🚀

In [ ]:
# Celda final de verificación y resumen
print("🎉 TAREA INTEGRADORA COMPLETADA")
print("=" * 50)
print("✅ Configuración del entorno")
print("✅ Implementación de clases (GridWorld + Q-Learning)")
print("✅ Entrenamiento y recolección de datos")
print("✅ Análisis estadístico")
print("✅ Visualizaciones comprehensivas")
print("✅ Evaluación final del rendimiento")
print("\n📝 PENDIENTE POR COMPLETAR:")
print("⏳ Análisis interpretativo (mínimo 200 palabras)")
print("⏳ Verificación de lista de control")
print("⏳ Nomenclatura correcta del archivo")
print("\n🏆 ¡Buena suerte con su entrega!")

# Información del estudiante (completar)
print("\n" + "="*30)
print("INFORMACIÓN DEL ESTUDIANTE")
print("="*30)
print("Nombre: [COMPLETAR]")
print("Apellido: [COMPLETAR]")
print("Fecha de entrega: [COMPLETAR]")
print("Nombre del archivo: RL_Python_[Apellido]_[Nombre].ipynb")